In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import time
import json
import requests
import math
from tqdm import tqdm
import random
import uuid

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException


from bs4 import BeautifulSoup
from time import gmtime, strftime

import folium
from folium.plugins import MiniMap
from folium.plugins import HeatMap

import mysql.connector

In [ ]:
projectdb = mysql.connector.connect(
    host="",
    user="master",
    password="",
    database="edaproject"
)

cursor = projectdb.cursor()

sql_insert_query = """
INSERT INTO SCHOOL_INFO (ID, SIDO, GU, NAME, STATUS, SCHOOLTYPE, FOUNDTYPE, FOUND_DATE, MAIN_TEL, FAX_TEL, ADDRESS,
TOTAL_STUDENTS, MALE_STUDENTS, FEMALE_STUDENTS, TOTAL_TEACHER, MALE_TEACHER, FEMALE_TEACHER, EX_AREA, KESPA,
OFFICE_TEL
, CLASSROOM_TEL)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

### 지방교육재정알리미 사이트 크롤링

In [ ]:
projectdb = mysql.connector.connect(
    host="",
    user="master",
    password="",
    database="edaproject"
)

cursor = projectdb.cursor()

sql_insert_query = """
INSERT INTO SCHOOL_INFO_1 (ID, SIDO, GU, NAME, SCHOOLTYPE, SCHOOLLEVEL , FOUNDDATE, KESPA, MAIN_TEL, ADDRESS,
TOTAL_STUDENTS)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""


path = '/Users//dev/driver/chromedriver-mac-arm64/'
driver_path = path + 'chromedriver'
driver = webdriver.Chrome(service = Service(driver_path))
action = ActionChains(driver)
url = 'https://eduinfo.go.kr/portal/theme/schNmlStatusPage.do#none'
driver.set_window_size(900, 1080)
driver.get(url)
driver.implicitly_wait(5)

sido_box = driver.find_element(By.ID, 'eduOffcDivCd')
sido_list = sido_box.find_elements(By.TAG_NAME, 'option')[13:14]

for each_sido in sido_list:
    sido_box.send_keys(each_sido.text)    
    time.sleep(1)
    gu_box = driver.find_element(By.ID, 'sggCd')
    select_all = gu_box.find_element(By.TAG_NAME, 'option')
    gu_box.send_keys(select_all.text)

    ActionChains(driver).move_to_element(driver.find_element(By.ID, 'search_btn')).perform()
    driver.find_element(By.ID, 'search_btn').click()
    time.sleep(5)

    school_list = driver.find_element(By.ID, 'schList').find_elements(By.TAG_NAME, 'li')
    for each_school in school_list:
        school_list = driver.find_element(By.ID, 'schList').find_elements(By.TAG_NAME, 'li')
        ActionChains(driver).move_to_element(each_school).perform()
        each_school.click()

        tab_content = driver.find_element(By.CLASS_NAME, 'tab_content')
        
        try:
                school_name = tab_content.find_element(By.ID, 'titleSchlNm').text
        except:
             school_name = ''
        try:
                school_type = tab_content.find_element(By.ID, 'tableEstbDivCd').text
        except:
               school_type = ''
        try:
                school_level = tab_content.find_element(By.ID, 'tableSchlKndCd').text
        except:
               school_level = ''
        try:
                kespa = tab_content.find_element(By.ID, 'tableRgEduOffcDiv').text
        except:
               kespa = ''
        try:
                found_date = tab_content.find_element(By.ID, 'tableOpenDate').text.replace('-','')
                if found_date == '':
                        found_date = '18000101'
        except:
                found_date = '18000101'
        try:
                main_tel = tab_content.find_element(By.ID, 'tableUserTel').text.split(',')[0]
        except:
               main_tel = ''
        try:
                address = tab_content.find_element(By.ID, 'tableSchulRdnma').text
        except:
               address = ''
        try:
                sido = address.split(' ')[0]
        except:
               sido = ''
        try:
                gu = address.split(' ')[1]
        except:
               gu = ''
        try:
                student_num = driver.find_element(By.ID, 'sheetTable').find_elements(By.TAG_NAME,'tr')[3].text.split(' ')[-1].replace(',','')
        except:
               student_num = '0'
               
        uuid_value_school_name = uuid.uuid3(uuid.NAMESPACE_DNS, school_name)
        school_id = str(uuid_value_school_name)[-5:] + str(random.randrange(10000,99999))
        
        data = (school_id, sido, gu, school_name, school_type, school_level, 
                found_date, kespa, main_tel, 
                address, student_num)
        
        cursor.execute(sql_insert_query, data)
        projectdb.commit()



### 학교알리미 사이트 크롤링

In [ ]:
def generate_sequence(number):
    result = []
    increment = 4
    
    while increment < number:
        result.append(increment)
        increment += 4
    return result

def GetSchoolInfo(info_list):

    # 가져올 데이터

    si = each_sido
    gu = each_sigungu
    
    school_name = driver.find_element(By.TAG_NAME, 'p').text.split(' ')[0][1:]

    uuid_value = uuid.uuid3(uuid.NAMESPACE_DNS, school_name)
    school_id = str(uuid_value.int)[-6:]  + str(random.randrange(100,999))
    school_id = int(school_id)
    try:
        status = driver.find_element(By.CLASS_NAME, 'closed_school').text
    except:
        status = ''

    school_type = info_list[1].split(':')[1].split(' ')[0]
    found_type = info_list[1].split(':')[2].split(' ')[0]
    found_date = info_list[1].split(':')[-1].replace(' ','').replace('년','').replace('월','').replace('일','')
    main_tel = info_list[2].split(' ')[1][1:].split(',')[0]

    fax = info_list[2].split(' ')[-1][1:]
    address = info_list[3].split(':')[1]

    total_student = info_list[4].split(' ')[2].replace('명','').replace(',','')
    male_student = info_list[4].split(' ')[4].replace('명','').replace(',','')
    female_student = info_list[4].split(' ')[7].replace('명','')[:-1].replace(',','')

    total_teacher = info_list[5].split(' ')[2].replace('명','').replace(',','')
    male_teacher = info_list[5].split(' ')[4].replace('명','').replace(',','')
    female_teacher = info_list[5].split(' ')[7].replace('명','')[:-1].replace(',','')

    exs_area = info_list[6].split(':')[1][:-1]
    kespa = info_list[7].split(':')[1]
    office_tel = info_list[8].split(' ')[1][1:]
    classroom_tel = info_list[8].split(' ')[-1][1:]
    # url = info_list[9].split(' ')[-1]
    #모든 데이터 한 리스트로 정리
    data = (school_id, si, gu, school_name, status, school_type, found_type,
            found_date, main_tel,fax,address,int(total_student),int(male_student),
            int(female_student),int(total_teacher),int(male_teacher),
            int(female_teacher),int(exs_area),kespa,office_tel,classroom_tel)

    return data
 
def SidoCircle():

    sido_select_box = driver.find_element(By.ID, 'mLevel2')
    sido_select_box.send_keys(each_sido)
    
    sigungu_select_box = driver.find_element(By.ID, 'mLevel3')
    sigungu_list = sigungu_select_box.get_attribute('innerText').split('\n')[1:]

    for sigungu_idx, each_sigungu in enumerate(sigungu_list):
        sigungu_select_box.send_keys(each_sigungu)            

        school_select_box = driver.find_element(By.ID, 'mLevel4')
        school_list = school_select_box.get_attribute('innerText').split('\n')[1:]
        for school_idx , each_school in enumerate(school_list):
            temp = []
            school_select_box.send_keys(each_school)
            driver.find_element(By.ID,'mobileSearchButton').click()
            driver.switch_to.window(driver.window_handles[1])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'schoolInfo1')))
            #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'owl-stage-outer')))
            info_list = driver.find_element(By.CLASS_NAME, 'basic_data').text
            info_list = info_list.split('\n')
            time.sleep(0.5)
            data = GetSchoolInfo(info_list)
            if data[4] =='폐':
                continue
            else:
                try:
                    cursor.execute(sql_insert_query, data)
                    projectdb.commit()
                except:
                    print(data)
            driver.switch_to.window(driver.window_handles[0])   

def GuCircle():
    sigungu_select_box = driver.find_element(By.ID, 'mLevel3')
    sigungu_select_box.send_keys(each_sigungu)  
    school_select_box = driver.find_element(By.ID, 'mLevel4')
    school_list = school_select_box.get_attribute('innerText').split('\n')[1:]
    for school_idx , each_school in enumerate(school_list):
        temp = []
        school_select_box.send_keys(each_school)
        driver.find_element(By.ID,'mobileSearchButton').click()
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'schoolInfo1')))

        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'owl-stage-outer')))
        info_list = driver.find_element(By.CLASS_NAME, 'basic_data').text
        info_list = info_list.split('\n')
        time.sleep(0.5)
        data = GetSchoolInfo(info_list)
        if data[4] =='폐':
            continue
        else:
            try:
                cursor.execute(sql_insert_query, data)
                projectdb.commit()
            except:
                print(data)
        driver.switch_to.window(driver.window_handles[0])   

def SchoolCircle():
    sido_select_box = driver.find_element(By.ID, 'mLevel2')
    sido_select_box.send_keys(each_sido)
    school_select_box = driver.find_element(By.ID, 'mLevel4')
    school_list = school_select_box.get_attribute('innerText').split('\n')[1:]
    for school_idx , each_school in enumerate(school_list):
        temp = []
        school_select_box.send_keys(each_school)
        driver.find_element(By.ID,'mobileSearchButton').click()
        driver.switch_to.window(driver.window_handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'schoolInfo1')))

        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'owl-stage-outer')))
        info_list = driver.find_element(By.CLASS_NAME, 'basic_data').text
        info_list = info_list.split('\n')
        time.sleep(0.5)
        data = GetSchoolInfo(info_list)
        if data[4] =='폐':
                continue
        else:
            try:
                cursor.execute(sql_insert_query, data)
                projectdb.commit()
            except:
                print(data)
        driver.switch_to.window(driver.window_handles[0])

def Refresh():
    path = '/Users//dev/driver/chromedriver-mac-arm64/'
    driver_path = path + 'chromedriver'
    driver = webdriver.Chrome(service = Service(driver_path))
    url = 'https://www.schoolinfo.go.kr/ei/ss/pneiss_a03_s0.do'
    driver.set_window_size(800, 1080)
    driver.get(url)
    driver.execute_script("window.scrollTo(0, 120)")
    return driver


path = '/Users//dev/driver/chromedriver-mac-arm64/'
driver_path = path + 'chromedriver'
driver = webdriver.Chrome(service = Service(driver_path))
action = ActionChains(driver)
url = 'https://www.schoolinfo.go.kr/ei/ss/pneiss_a03_s0.do'
driver.set_window_size(800, 1080)
driver.get(url)
# 사이즈 기준으로 html 구조가 달라짐
# 작은 사이즈로 할 경우 더 크롤링 하기 쉬움
driver.execute_script("window.scrollTo(0, 120)")

#임시 저장
driver.find_element(By.CLASS_NAME, 'radio-group').find_elements(By.TAG_NAME, 'li')[1].click() # [0] = 초등학교 
sido_select_box = driver.find_element(By.ID, 'mLevel2')
sido_list = sido_select_box.get_attribute('innerText').split('\n')[1:] # 각 시마다 불러오기
for sido_idx , each_sido in enumerate(sido_list):
    if each_sido == '세종특별자치시':
        driver.close()
        time.sleep(random.randrange(10,15))
        ### Refresh ###
        path = '/Users//dev/driver/chromedriver-mac-arm64/'
        driver_path = path + 'chromedriver'
        driver = webdriver.Chrome(service = Service(driver_path))
        url = 'https://www.schoolinfo.go.kr/ei/ss/pneiss_a03_s0.do'
        driver.set_window_size(800, 1080)
        driver.get(url)
        driver.execute_script("window.scrollTo(0, 120)")
        driver.execute_script("window.scrollTo(0, 120)")
        driver.find_element(By.CLASS_NAME, 'radio-group').find_elements(By.TAG_NAME, 'li')[1].click()
        each_school_level.click()
        ### Refresh ###
        time.sleep(2)
        #세종 특별
        SchoolCircle()
        
    else:
        driver.close()
        time.sleep(random.randrange(10,15))
        ### Refresh ###
        path = '/Users//dev/driver/chromedriver-mac-arm64/'
        driver_path = path + 'chromedriver'
        driver = webdriver.Chrome(service = Service(driver_path))
        url = 'https://www.schoolinfo.go.kr/ei/ss/pneiss_a03_s0.do'
        driver.set_window_size(800, 1080)
        driver.get(url)
        driver.execute_script("window.scrollTo(0, 120)")
        driver.execute_script("window.scrollTo(0, 120)")
        driver.find_element(By.CLASS_NAME, 'radio-group').find_elements(By.TAG_NAME, 'li')[1].click()
        ### Refresh ###
        time.sleep(2)
        sido_select_box = driver.find_element(By.ID, 'mLevel2')
        sido_select_box.send_keys(each_sido)
        sigungu_select_box = driver.find_element(By.ID, 'mLevel3')
        sigungu_list = sigungu_select_box.get_attribute('innerText').split('\n')[1:]
        if len(sigungu_list) > 6:
            print(each_sido,' has many gu')
            crewling_break_idx = generate_sequence(len(sigungu_list))
            for sigungu_idx, each_sigungu in enumerate(sigungu_list):
                ### 매 6번째 구마다 재시작
                if sigungu_idx in crewling_break_idx:
                    print('끈어서 다시')
                    ### Refresh ###
                    path = '/Users//dev/driver/chromedriver-mac-arm64/'
                    driver_path = path + 'chromedriver'
                    driver = webdriver.Chrome(service = Service(driver_path))
                    url = 'https://www.schoolinfo.go.kr/ei/ss/pneiss_a03_s0.do'
                    driver.set_window_size(800, 1080)
                    driver.get(url)
                    driver.execute_script("window.scrollTo(0, 120)")
                    driver.execute_script("window.scrollTo(0, 120)")
                    driver.find_element(By.CLASS_NAME, 'radio-group').find_elements(By.TAG_NAME, 'li')[1].click()
                    ### Refresh ###                    
                    driver.refresh()
                    time.sleep(2)
                    sido_select_box = driver.find_element(By.ID, 'mLevel2')
                    sido_select_box.send_keys(each_sido)
                    GuCircle() 
                else:
                    print('Not a Refresh Gu')
                    GuCircle()
        else:
            print('기본')
            SidoCircle()
